## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [108]:
import pandas as pd
from google.colab import files
import joblib



In [2]:
#path =r'C:\Users\DELL\Desktop.tsv'


In [4]:
df =pd.read_csv('train.tsv',sep='\t')


In [5]:
csv_data = df.to_csv(index=False)

In [6]:
import os


In [7]:
file_path = os.path.join(os.getcwd(), 'train.csv')


In [8]:
with open(file_path, 'w') as f:
  f.write(csv_data)

In [9]:
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [10]:
df.shape

(99999, 2)

In [11]:
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [12]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [13]:
###Drop Nan Values
df=df.dropna()


In [14]:
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [15]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [16]:
y=df['label']

In [17]:
#Y=df.drop('label',axis=1)

In [18]:
y.shape

(99999,)

In [19]:
y.head

<bound method NDFrame.head of 0        Positive
1        Positive
2        Positive
3        Positive
4        Positive
           ...   
99994    Negative
99995    Negative
99996    Negative
99997    Negative
99998    Negative
Name: label, Length: 99999, dtype: object>

In [20]:
X.shape

(99999, 1)

In [21]:
X.head

<bound method NDFrame.head of                                                     text
0      ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1      أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2      هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3      خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4      ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...                                                  ...
99994  معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995  لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996  كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997  مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...
99998  لن ارجع إليه مرة اخرى . قربه من البحر. المكان ...

[99999 rows x 1 columns]>

In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'2.12.0'

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [25]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [26]:
messages=X.copy()

In [27]:
messages

,text
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...,...
99994,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...


In [28]:
messages['text'][0]

'ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم'

In [29]:
#messages.reset_index(inplace=True)

In [30]:
import nltk
import re
from nltk.corpus import stopwords

In [39]:
nltk.download('stopwords-ar')


[nltk_data] Error loading stopwords-ar: Package 'stopwords-ar' not
[nltk_data]     found in index


False

In [40]:
stopwords_arabic = stopwords.words('arabic')

In [41]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
lemmatizer = WordNetLemmatizer()


In [43]:
corpus = [1]


In [44]:
# Loop over each message in the dataset
for i in range(0, len(messages)):
    # Remove non-alphabetic characters using regular expressions
    review = re.sub('[^ء-ي]', ' ', messages['text'][i])
     # Convert text to lowercase
    review = review.lower()
    # Split text into words
    review = review.split()
    # Perform lemmatization and stopword removal on each word
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('arabic')]
       # Join the lemmatized words back into a single string
    review = ' '.join(review) 
        # Add the preprocessed text to the corpus list
    corpus.append(review)


In [45]:
## Dataset Preprocessing
#from nltk.stem.porter import PorterStemmer ##stemming purpose
#ps = PorterStemmer()
#corpus = []
#for i in range(0, len(messages)):
#    review = re.sub('[^ء-ي]', ' ', messages['text'][i])
#    review = review.lower()
#    review = review.split()
    
#    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#    review = ' '.join(review)
#    corpus.append(review)
##

In [ ]:
corpus


In [47]:
corpus[1]

'ممتاز نوعا النظافة والموقع والتجهيز والشاطيء المطعم'

In [ ]:
onehot_repr = [one_hot(str(words), voc_size) for words in corpus]
onehot_repr

In [49]:
len(corpus)

100000

In [50]:
onehot_repr[1]

[1849, 163, 1726, 622, 2034, 252, 4924]

### Embedding Representation

In [51]:
sent_length=10
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1189    0    0 ...    0    0    0]
 [1849  163 1726 ...    0    0    0]
 [1909 3728 3732 ... 2889  980  605]
 ...
 [4471 1159 4439 ... 3725  860 3869]
 [3839 3658  965 ... 1819 2864 2702]
 [3027 2449 1288 ...  337 1256 4503]]


In [52]:
len(embedded_docs)

100000

In [53]:
embedded_docs[0]

array([1189,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [54]:
embedded_docs[0]

array([1189,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [55]:
from keras.layers import Dropout


In [56]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 10, 40)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
__________________________________________________

In [57]:
##from keras.layers import Dropout

##embedding_vector_features = 40
##model = Sequential()
##model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
##model.add(LSTM(100, dropout=0.2))
##model.add(Dense(1, activation='softmax'))

##model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [58]:
from keras.callbacks import LearningRateScheduler

def lr_schedule(epoch):
    lr = 0.001
    if epoch > 5:
        lr *= 0.1
    elif epoch > 2:
        lr *= 0.5
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)


In [59]:
embedded_docs.shape

(100000, 10)

In [60]:
import numpy as np
X_final1=np.array(embedded_docs)
y_final=np.array(y)

In [61]:
X_final1

array([[1189,    0,    0, ...,    0,    0,    0],
       [1849,  163, 1726, ...,    0,    0,    0],
       [1909, 3728, 3732, ..., 2889,  980,  605],
       ...,
       [4471, 1159, 4439, ..., 3725,  860, 3869],
       [3839, 3658,  965, ..., 1819, 2864, 2702],
       [3027, 2449, 1288, ...,  337, 1256, 4503]], dtype=int32)

In [62]:
y_final

array(['Positive', 'Positive', 'Positive', ..., 'Negative', 'Negative',
       'Negative'], dtype=object)

In [63]:
y_final[y_final=="Positive"] = 1.0
y_final[y_final=="Negative"] = 0.0
y_final[y_final=="Mixed"] = 1.0


In [64]:
y_final

array([1.0, 1.0, 1.0, ..., 0.0, 0.0, 0.0], dtype=object)

In [65]:
embedded_docs

array([[1189,    0,    0, ...,    0,    0,    0],
       [1849,  163, 1726, ...,    0,    0,    0],
       [1909, 3728, 3732, ..., 2889,  980,  605],
       ...,
       [4471, 1159, 4439, ..., 3725,  860, 3869],
       [3839, 3658,  965, ..., 1819, 2864, 2702],
       [3027, 2449, 1288, ...,  337, 1256, 4503]], dtype=int32)

In [66]:
X_final = X_final1[1:, :]

In [67]:
X_final.shape,y_final.shape

((99999, 10), (99999,))

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [69]:
X_train.shape

(66999, 10)

In [70]:
y_train.shape

(66999,)

In [71]:
X_test.shape

(33000, 10)

In [72]:
y_test.shape

(33000,)

In [73]:
X_train[1]

array([4127, 3780,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [74]:
y_train[1]

1.0

In [75]:
y_test[1]

0.0

In [76]:
X_test[1]

array([3763, 1189, 4286, 2073, 1555, 2507, 4854,    0,    0,    0],
      dtype=int32)

In [77]:
X_test = X_test.astype(np.float32)


In [78]:
X_testt = tf.constant(X_test)


In [79]:
X_testt

<tf.Tensor: shape=(33000, 10), dtype=float32, numpy=
array([[2762., 3177., 2515., ..., 4230., 1076.,   28.],
       [3763., 1189., 4286., ...,    0.,    0.,    0.],
       [2602., 4978.,  610., ..., 4331.,  315.,  204.],
       ...,
       [2153., 4884., 1061., ...,    0.,    0.,    0.],
       [3065., 2127., 1229., ...,    0.,    0.,    0.],
       [4307.,  935.,   52., ..., 1691., 3073., 1929.]], dtype=float32)>

In [80]:
print(y_test.dtype)


object


In [81]:
y_test

array([1.0, 0.0, 1.0, ..., 1.0, 1.0, 1.0], dtype=object)

In [82]:
y_test = y_test.astype(np.float32)


In [83]:
y_testt = tf.constant(y_test)


In [84]:
y_testt

<tf.Tensor: shape=(33000,), dtype=float32, numpy=array([1., 0., 1., ..., 1., 1., 1.], dtype=float32)>

In [85]:
y_train = y_train.astype(np.float32)


In [86]:
y_trainn = tf.constant(y_train)


In [87]:
y_trainn

<tf.Tensor: shape=(66999,), dtype=float32, numpy=array([1., 1., 0., ..., 0., 1., 1.], dtype=float32)>

In [88]:
X_train = X_train.astype(np.float32)


In [89]:
X_trainn = tf.constant(X_train)


In [90]:
X_trainn

<tf.Tensor: shape=(66999, 10), dtype=float32, numpy=
array([[2669.,  684., 4801., ..., 3203., 3765., 3963.],
       [4127., 3780.,    0., ...,    0.,    0.,    0.],
       [3805.,  616.,  480., ...,    0.,    0.,    0.],
       ...,
       [1130., 4481., 3977., ..., 2577., 2167., 3752.],
       [1480., 2602.,  362., ..., 2622., 1343., 3378.],
       [2982., 1347., 4709., ...,  421., 1727., 4698.]], dtype=float32)>

In [91]:
y_trainn

<tf.Tensor: shape=(66999,), dtype=float32, numpy=array([1., 1., 0., ..., 0., 1., 1.], dtype=float32)>

In [92]:
lennnn=y_trainn

In [93]:
print(lennnn)

tf.Tensor([1. 1. 0. ... 0. 1. 1.], shape=(66999,), dtype=float32)


In [94]:
test_loss, test_acc = model.evaluate(X_test, y_test)

1032/1032 [==============================] - 9s 3ms/step - loss: 0.6932 - accuracy: 0.4879


In [95]:

## Model Training
model.fit(X_trainn,y_trainn,validation_data=(X_testt,y_testt),epochs=10,batch_size=32)

Epoch 1/10
2094/2094 [==============================] - 43s 17ms/step - loss: 0.5610 - accuracy: 0.7103 - val_loss: 0.5420 - val_accuracy: 0.7204
Epoch 2/10
2094/2094 [==============================] - 18s 9ms/step - loss: 0.5166 - accuracy: 0.7390 - val_loss: 0.5486 - val_accuracy: 0.7153
Epoch 3/10
2094/2094 [==============================] - 22s 10ms/step - loss: 0.4910 - accuracy: 0.7560 - val_loss: 0.5567 - val_accuracy: 0.7112
Epoch 4/10
2094/2094 [==============================] - 19s 9ms/step - loss: 0.4637 - accuracy: 0.7732 - val_loss: 0.5752 - val_accuracy: 0.7129
Epoch 5/10
2094/2094 [==============================] - 18s 9ms/step - loss: 0.4361 - accuracy: 0.7919 - val_loss: 0.6012 - val_accuracy: 0.6988
Epoch 6/10
2094/2094 [==============================] - 20s 10ms/step - loss: 0.4078 - accuracy: 0.8079 - val_loss: 0.6222 - val_accuracy: 0.6976
Epoch 7/10
2094/2094 [==============================] - 18s 9ms/step - loss: 0.3832 - accuracy: 0.8234 - val_loss: 0.6531 - val

### Model Training

### Adding Dropout 

### Performance Metrics And Accuracy

In [189]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

In [190]:
new_sentence = "جيد ممتاز "


In [191]:
corpus = []


In [192]:
review = re.sub('[^ء-ي]', ' ', new_sentence)
review = review.lower()
review = review.split()

In [193]:
from nltk.stem.porter import PorterStemmer


In [194]:
ps = PorterStemmer()


In [195]:
review = [ps.stem(word) for word in review if not word in stopwords.words('arabic')]
review = ' '.join(review)
corpus.append(review)
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [196]:
new_embedded=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [197]:
# Make predictions
prediction = model.predict(new_embedded)[0][0]
if prediction < 0.5:
    print("Negative review")
else:
    print("Positive review")

1/1 [==============================] - 0s 19ms/step
Positive review


In [198]:
prediction

0.8061677